<a href="https://colab.research.google.com/github/HassanSherwani/Product_Purchase_Frequency/blob/master/Product_Freq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement:

How often a certain product has been sold in the past days.

#1)- Importing key modules

In [0]:
# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import sys

In [3]:
! pip install turicreate

    100% |████████████████████████████████| 87.4MB 249kB/s 
    100% |████████████████████████████████| 2.7MB 11.3MB/s 
    100% |████████████████████████████████| 23.8MB 1.7MB/s 
mxnet 1.1.0.post0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.16.2 which is incompatible.
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
import turicreate as tc

# 2)-Loading Data

In [0]:
url = 'https://raw.githubusercontent.com/HassanSherwani/Product_Purchase_Frequency/master/20190207_transactions%20.json'

In [0]:
transactions = pd.read_json(url, lines= True)

# 3)-Exploring dataset

In [7]:
transactions.head()

,id,products
0,0,"[185, 30, 77, 188, 78, 125, 45, 155, 241, 229,..."
1,1,"[119, 148, 108, 34, 157, 82, 113, 45, 165]"
2,2,"[173, 103, 229, 240]"
3,3,[91]
4,4,"[175, 192, 54, 172]"


In [8]:
transactions.shape

(2500, 2)

In [9]:
print(transactions['products'][1415])

[250, 236, 242, 229, 92, 2, 71, 172, 109, 247, 171, 209, 90, 139, 188, 191, 145, 214, 216, 237]


In [10]:
transactions.info() # checking missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 2 columns):
id          2500 non-null int64
products    2500 non-null object
dtypes: int64(1), object(1)
memory usage: 39.1+ KB


In [11]:
transactions.describe()

,id
count,2500.00000
mean,1249.50000
std,721.83216
min,0.00000
25%,624.75000
50%,1249.50000
75%,1874.25000
max,2499.00000


# 4)- Adding Features:

### 4.1)-Create a separate dataframe for recommending users

In [0]:
customers=transactions['id']

In [0]:
customers.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [0]:
import random
random.shuffle(customers) 

 To get random values of customer ids. This would help us in avoiding sample biaseness in our modeling.

In [0]:
cust_2_rec=customers[:1000]

In [17]:
cust_2_rec.head()

0    1448
1    1010
2    1102
3    1168
4     287
Name: id, dtype: int64

### 4.2)- break down each list of items in the products column into rows and count the number of products bought by a user

In [18]:
pd.melt(transactions.head(2).set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,id,productId,purchase_count
0,1010,34.0,1
1,1010,45.0,1
2,1010,82.0,1
3,1010,108.0,1
4,1010,113.0,1
5,1010,119.0,1
6,1010,148.0,1
7,1010,157.0,1
8,1010,165.0,1
9,1448,30.0,1


# 5- Creating features for user_id,product_id and purchase count

### 5a)-One for purchase count

In [19]:
s=time.time()

data = pd.melt(transactions.set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.01 minutes


In [20]:
data.shape

(24811, 3)

In [21]:
data.head()

,id,productId,purchase_count
0,0,8,1
1,0,9,1
2,0,19,1
3,0,26,2
4,0,48,1


In [0]:
prod_freq=data['productId'].value_counts(dropna=False)

In [0]:
prod_freq.index.name
prod_freq.index.name='prod_ID'

In [0]:
def product_frequency(prod_ID):
    if prod_ID not in prod_freq.index:
        print('Product not found.')
        return prod_ID
    return prod_freq.loc[prod_ID]

In [25]:
prod_ID=int(input("enter a product ID to find frequency: "))

enter a product ID to find frequency: 248


In [26]:
print(product_frequency(prod_ID))

127


**This product frequency is not how each customer has purchased it. To add to our basic problem, we need to see how often a product has been purchased accross users. For this reason, I didn't end my assignment here.**

### 5-b)-Normalize item

we normalize purchase frequency of each item across users by first creating a user-item matrix 

In [0]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')

In [28]:
df_matrix.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The NaN tells us that the item represented by the column was not purchased in that specific transaction.

In [29]:
df_matrix.shape

(2378, 250)

In [30]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

df_matrix_norm.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
print(df_matrix_norm.shape)

(2378, 250)


### 5-c)- create a table for input to the modeling

In [0]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()

In [33]:
data_norm.head()

,id,productId,scaled_purchase_freq
32,33,1,0.0
75,78,1,0.0
83,86,1,0.0
87,90,1,0.0
93,96,1,0.0


In [34]:
data_norm.shape

(22530, 3)

### 5-d) a function for normalizing data

In [0]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()


we have normalized the items accoreding to their purchase history, from 0–1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item.

# 6)-Split train and test set

we have three dataframes with purchase counts(data), purchase dummy(data_dummy), and scaled purchase counts(data_norm).

In [36]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(19848, 3) (4963, 3)


In [0]:
#convert dataframe to SFrame
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

A tabular, column-mutable dataframe object that can scale to big data <br>
https://turi.com/products/create/docs/generated/graphlab.SFrame.html

In [38]:
train_data

id,productId,purchase_count
2301,207,1
632,38,1
1066,203,1
643,136,1
389,156,1
2309,183,1
1535,27,1
1328,218,1
790,74,1
1985,223,1


In [39]:
test_data

id,productId,purchase_count
2265,15,1
709,133,1
1936,246,1
409,107,1
2167,132,1
1579,194,1
250,165,1
2030,245,1
327,79,2
319,197,1


### 6.2)- Define a split_data function 

In [0]:
def split_data(data):

    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

### 6.3)-Apply for scaled/normalized purchase table

In [0]:
train_data_norm, test_data_norm = split_data(data_norm)

# 7)-Building Model

### 7.1)- Parameters to define field names for purchase count as target feature

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_count'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [43]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 19848 observations with 2351 users and 250 items.

Data prepared in: 0.051259s

19848 observations to process; with 250 unique items.

In [44]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 85601.8

+------+-----------+--------------------+------+
|  id  | productId |       score        | rank |
+------+-----------+--------------------+------+
| 1448 |    207    | 1.0952380952380953 |  1   |
| 1448 |     96    | 1.0845070422535212 |  2   |
| 1448 |    153    |        1.08        |  3   |
| 1448 |    201    | 1.0769230769230769 |  4   |
| 1448 |     44    | 1.0740740740740742 |  5   |
| 1448 |    112    | 1.0689655172413792 |  6   |
| 1448 |    150    | 1.0666666666666667 |  7   |
| 1448 |    152    | 1.0638297872340425 |  8   |
| 1448 |     85    | 1.0618556701030928 |  9   |
| 1448 |    138    | 1.0609756097560976 |  10  |
| 1010 |    207    | 1.0952380952380953 |  1   |
| 1010 |     96    | 1.0845070422535212 |  2   |
| 1010 |    153    |        1.08        |  3   |
| 1010 |    201    | 1.0769230769230769 |  4   |
| 1010 |     44    | 1.0740740740740742 |  5   |
| 1010 |    112    | 1.0689655172413792 |  6   |
| 1010 |    150    | 1.0666666666666667 |  7   |
| 1010 |    152    |

### Note 

Through this model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations.<br> These 30 records include 3 users and their recommended items, along with score and descending ranks.

## Checking most frequent items(as per assignment) 

In [45]:
 train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
207    1.095238
96     1.084507
153    1.080000
201    1.076923
44     1.074074
112    1.068966
150    1.066667
152    1.063830
85     1.061856
125    1.061728
138    1.060976
131    1.058824
26     1.058824
117    1.058824
63     1.056818
7      1.056180
247    1.055556
88     1.054795
181    1.054795
186    1.053333
Name: purchase_count, dtype: float64

**products 201,207,125,112,96 and 153 are the most popular (best-selling) across customers.**

### 7.2)- Applying 'scaled_purchase_freq' as target feature on model

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'scaled_purchase_freq'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [47]:
popularity_model_scaled = tc.popularity_recommender.create(train_data_norm, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 18024 observations with 2338 users and 227 items.

Data prepared in: 0.041173s

18024 observations to process; with 227 unique items.

In [48]:
popularity_recomm_4_scaled = popularity_model_scaled.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_scaled.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 53625.1

+------+-----------+---------------------+------+
|  id  | productId |        score        | rank |
+------+-----------+---------------------+------+
| 1448 |    201    | 0.09090909090909091 |  1   |
| 1448 |     26    |         0.08        |  2   |
| 1448 |    153    | 0.07894736842105263 |  3   |
| 1448 |     96    | 0.07792207792207792 |  4   |
| 1448 |    129    | 0.07575757575757576 |  5   |
| 1448 |    207    | 0.07058823529411765 |  6   |
| 1448 |     35    | 0.06756756756756757 |  7   |
| 1448 |    117    | 0.06578947368421052 |  8   |
| 1448 |    152    | 0.06521739130434782 |  9   |
| 1448 |    120    |  0.0641025641025641 |  10  |
| 1010 |    201    | 0.09090909090909091 |  1   |
| 1010 |     26    |         0.08        |  2   |
| 1010 |    153    | 0.07894736842105263 |  3   |
| 1010 |     96    | 0.07792207792207792 |  4   |
| 1010 |    129    | 0.07575757575757576 |  5   |
| 1010 |    207    | 0.07058823529411765 |  6   |
| 1010 |     35    | 0.06756756756756757 |  7   |


Great!.... We have all three models worked out. We have purchase frequency of each item customer-vice. <br>
Which one to trust and which one to discard? 

# 8)- Evaluate models

### 8.1)- Validation for Popularity Model on Purchase Counts

In [0]:
models_counts = [popularity_model]

In [0]:
model_names=['Popularity Model on Purchase Counts']

In [51]:
eval_counts = tc.recommender.util.compare_models(test_data, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/1923 queries. users per second: 46908.7


Precision and recall summary statistics by cutoff
+--------+----------------------+-----------------------+
| cutoff |    mean_precision    |      mean_recall      |
+--------+----------------------+-----------------------+
|   1    | 0.010400416016640688 | 0.0052839891373432715 |
|   2    | 0.01248049921996878  |  0.011018662968740966 |
|   3    | 0.012480499219968778 |  0.016337320159472996 |
|   4    | 0.01235049401976078  |  0.01967989830704342  |
|   5    | 0.013104524180967265 |  0.02660112753716492  |
|   6    | 0.012567169353440814 |  0.030775532608605938 |
|   7    | 0.012554787905801971 |  0.03651309544445272  |
|   8    | 0.012675507020280816 |  0.042276659320341026 |
|   9    | 0.012653839486912793 |  0.04711285276807897  |
|   10   | 0.012532501300051988 |  0.05238673038985049  |
+--------+----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.1430332375828223

Per User RMSE (best)
+------+------+-------+
|  id  | rmse | count |
+------+--

### 8.2)- Validation for Popularity Model on Scaled Purchase Counts

In [52]:
models_counts = [popularity_model_scaled]
model_names=['Popularity Model on Scaled Purchase Counts']
eval_counts_dummy = tc.recommender.util.compare_models(test_data_norm, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/1863 queries. users per second: 46801.1


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.010735373054213643 | 0.003973366050660732 |
|   2    | 0.011003757380568984 | 0.009091838560437581 |
|   3    | 0.011451064591161209 | 0.014267821997290597 |
|   4    | 0.011808910359634993 | 0.020445773585870176 |
|   5    | 0.012238325281803557 | 0.025384045190808495 |
|   6    | 0.012077294685990341 | 0.030264805868670613 |
|   7    | 0.011962272831838063 | 0.035968611814022534 |
|   8    | 0.01167471819645731  | 0.03922628632773558  |
|   9    | 0.011749269398222724 | 0.04426296551900406  |
|   10   | 0.01159420289855072  | 0.04762671574265782  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.15603104680197963

Per User RMSE (best)
+-----+------+-------+
|  id | rmse | count |
+-----+------+-------+
| 

 We tend to keep lowest rmse and it signifies least error in prediction. 



**So we will choose that as best model with rmse 0.147 i.e (in 8.1) for our final step.**

# 9) Submission

In [53]:
users_to_recommend = list(cust_2_rec)

final_model = tc.item_similarity_recommender.create(tc.SFrame(data), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_count')

Preparing data set.

Data has 24811 observations with 2378 users and 250 items.

Data prepared in: 0.034392s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 24.039ms                            | 100              | 250             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.03233s

In [54]:
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 51132.6

+------+-----------+----------------------+------+
|  id  | productId |        score         | rank |
+------+-----------+----------------------+------+
| 1448 |    248    | 0.027617092927296957 |  1   |
| 1448 |    168    | 0.025631868839263917 |  2   |
| 1448 |     23    | 0.024235220750172932 |  3   |
| 1448 |    214    | 0.023167447249094645 |  4   |
| 1448 |     32    | 0.02150735855102539  |  5   |
| 1448 |     61    | 0.02031291723251343  |  6   |
| 1448 |    243    | 0.020260564486185708 |  7   |
| 1448 |    126    | 0.02020918925603231  |  8   |
| 1448 |     63    | 0.020147351423899333 |  9   |
| 1448 |     74    | 0.01874724229176839  |  10  |
| 1010 |     23    | 0.027408692571851943 |  1   |
| 1010 |    214    | 0.025738835334777832 |  2   |
| 1010 |     80    | 0.02526219023598565  |  3   |
| 1010 |    112    | 0.025098118517133925 |  4   |
| 1010 |     2     | 0.02443495061662462  |  5   |
| 1010 |     40    | 0.024198671181996662 |  6   |
| 1010 |     17    | 0.02363376

### 9.2)- Checking most frequent items in final model

In [55]:
data.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
96     1.080808
207    1.076923
153    1.071429
201    1.069767
26     1.063158
152    1.060345
125    1.058824
112    1.058824
85     1.058824
131    1.057143
35     1.056180
63     1.054545
44     1.054545
129    1.053763
101    1.053763
117    1.052632
150    1.051724
122    1.051724
156    1.051546
120    1.050000
Name: purchase_count, dtype: float64

**RESULT: <BR>
products 96,207 , 153,201 and 26(and so on) are the most frequent selling items across customers.**

# 10)- Bonus Part- Recommending products to customers

In [56]:
# Create a csv output file
df_rec = recom.to_dataframe()
df_rec.head()

,id,productId,score,rank
0,1448,248,0.027617,1
1,1448,168,0.025632,2
2,1448,23,0.024235,3
3,1448,214,0.023167,4
4,1448,32,0.021507,5


In [57]:
print(df_rec.shape)

(10000, 4)


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates().sort_values('id').set_index('id')

In [59]:
recomendation = final_model.recommend(users=users_to_recommend, k=n_rec)

recommendations finished on 1000/1000 queries. users per second: 41837.5

In [0]:
df_rec = recomendation.to_dataframe()


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))

In [0]:
df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates() \
        .sort_values('id').set_index('id')

In [63]:
df_output.head()

,recommendedProducts
id,
3,50|41|159|245|170|85|147|183|75|45
4,3|78|108|87|84|236|125|227|223|146
5,11|126|65|229|24|2|242|85|79|181
6,21|69|246|87|36|125|90|77|211|126
7,41|9|125|131|207|33|121|28|137|214


In [0]:
# define recommendation function
def customer_recomendation(id):
    if id not in df_output.index:
        print('Customer not found.')
        return id
    return df_output.loc[id]

In [65]:
customer_recomendation(48)

Customer not found.


48

In [66]:
customer_recomendation(2438)

Customer not found.


2438

In [67]:
customer_recomendation(368)

recommendedProducts    9|211|152|2|36|113|245|166|77|151
Name: 368, dtype: object